In [ ]:
#pip install langchain-community pypdf

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [4]:
print(documents)

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'), Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')]


In [5]:
for doc in documents:
    print(doc.page_content)


Dogs are great companions, known for their loyalty and friendliness.
Cats are independent pets that often enjoy their own space.


In [7]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "Chapter 01.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

19


In [8]:
print(f"{docs[0].page_content[:200]}\n")
print(docs[0].metadata)

IN2311:Operating systems
Roshani Wijesuriya

{'producer': 'Microsoft® PowerPoint® LTSC', 'creator': 'Microsoft® PowerPoint® LTSC', 'creationdate': '2024-10-12T10:35:42+05:30', 'title': '', 'author': 'Roshani Wijesuriya', 'moddate': '2024-10-12T10:35:42+05:30', 'source': 'Chapter 01.pdf', 'total_pages': 19, 'page': 0, 'page_label': '1'}
